In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from google.colab import files

In [ ]:
model = MobileNetV2(weights='imagenet')

14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
uploaded = files.upload()

for file_name in uploaded.keys():
    img_path = file_name
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    preds = model.predict(img_array)
    decoded = decode_predictions(preds, top=3)[0]

    print("Top Predictions:")
    for i, (code, label, prob) in enumerate(decoded):
        print(f"{i+1}: {label} ({prob*100:.2f}%)")

top_prediction = decoded[0][1].lower()

Saving pizza.jpeg to pizza.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Top Predictions:
1: pizza (25.87%)
2: confectionery (8.07%)
3: honeycomb (6.25%)


In [ ]:
uploaded = files.upload()  # Upload calories.csv here

df = pd.read_csv('calories.csv')
calorie_dict = dict(zip(df['Food'].str.lower(), df['Calories']))

Saving calories.csv to calories.csv


In [ ]:
if top_prediction in calorie_dict:
    estimated_calories = calorie_dict[top_prediction]
    print(f"\n🍽️ This looks like: {top_prediction}")
    print(f"🔥 Estimated Calories: {estimated_calories} kcal")
else:
    print(f"\n🤷‍♀️ '{top_prediction}' not found in calorie dictionary.")


🍽️ This looks like: pizza
🔥 Estimated Calories: 285 kcal
